Realizando a instalação do groq para poder fazer a importação da classe Groq

In [6]:
%pip install groq

Inserindo chave de api como variável de ambiente

In [7]:
import os
os.environ ["GROQ_API_KEY"] = "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"

Iniciando Cliente que vai receber a classe Groq para poder utilizar o modelo especiicado em model abaixo, em chat completion temos a instância client.chat.completions.create, onde pode ser inserido uma frase do usuário, a qual o modelo vai responder, com base no que foi treinado de forma direta, ou seja, ele vai somente explicar a importância de modelos rápidos de linguagem.

In [8]:
import os
from groq import Groq # Classe para interagir com API Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

# Lista de mensagens que a API recebe
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are artificial intelligence (AI) systems designed to process and understand human language quickly and efficiently. These models have become increasingly important in recent years due to their wide range of applications and benefits. Here are some reasons why fast language models are crucial:

1. **Improved User Experience**: Fast language models enable applications to respond quickly to user queries, providing a seamless and engaging user experience. This is particularly important for conversational AI systems, such as chatbots, virtual assistants, and voice assistants.
2. **Real-time Processing**: Fast language models can process and analyze large amounts of text data in real-time, allowing for applications such as sentiment analysis, entity recognition, and intent detection. This enables businesses to respond promptly to customer feedback, detect trends, and make data-driven decisions.
3. **Efficient Information Retrieval**: Fast language models can quickly retr

No trecho abaixo temos uma classe agente, formada por três funções, a primeira sendo _init_ a qual é responsável por fazer a comunicação entre a LLM (llama-3.3-70b-versatile) por meio do client, onde inserimos os nossos dados para interagir com a API do Grok, os comandos que ela deve seguir que são dados no system prompt, o qual vai coordenar se o agente deve agir como um agente ReAct, ou só retornar respostas sobre que t6em haver com a empresa y, entre outros casos e o histórico de mensagens que ela vai usar para separar as etapas durante seu processo de pensar, agir, pausar e observar, além de que com esse histórico somos capazes de acompanhar como o agente faz seu racíocínio. Já a função _call_ é responsável por realizar uma chamada toda vez que o agente é executado e dado que há mensagens essas são anexadas na lista que foi declarada em _init_ e por fim a função execute é definido o modelo que vai ser usado, como também temos o retorno do objeto que nos mostra as mensagens que ele recebeu.



In [9]:
class Agent:
  def __init__(self, client,system: str = "") -> None:
    self.client = client
    self.system = system
    self.messages: list = [] # Lista de mensagens será armazenado na memória do nosso agente
    if self.system: # Se houver mensagens do sistema será anexado aqui
        self.messages.append({"role": "system", "content": system})

  def __call__(self, message=""):
    if message:
        self.messages.append({"role": "system", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result


  def execute(self):
    chat_completion = client.chat.completions.create(
        messages=self.messages,
        model="llama-3.3-70b-versatile",)
    return chat_completion.choices[0].message.content

O trecho abaixo se trata do system prompt que como já foi dito antes fala de com o o agente deve proceder quando for fazer alguma coisa, onde o AgentReact vai  responder com um pensamento, seguido de uma ação e depois pausar em uma observação após pausar ele vai executar uma ferramenta e então retornar a observação e obter a resposta final do agente, é bastante importante ele retornar e pausar para que possa ser feita a análise, as ações disponíveis aqui são as tools que ele vai poder usar para responder a querry informada pelo usuário. No caso do agente abixo, ele possui duas ações disponíveis: sendo a primeira a possibilidade de poder realizar cálculos podendo usar o float do python se achar necessário e uma ação de poder obter a massa de um planeta, ambas as ações são seguidas de um exemplo de como funcionam com o passo a passo de um input do user, pensamento e ação do agente, quando ele deve realizar a pausa, que ele vai ser chamado novamente e quando parar que é o momento que ele chegou na resposta final.

In [10]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

As funções abaixo remetem as ferramentas que foram informadas no system prompt que podiam ser usadas nas ações do agente, temos a calculate para realizar os cálculos e a get_planet_mass que retorna para cada nome de planeta, o qual foi inserido no input do usuário, sua respectiva massa, a qual pode ser usada para repsonder pergunatas tais como: qual a masssa da terra ou qual a massa da terra vezes cinco.

In [11]:
def calculate(operation):
  return eval(operation)
# Pesquisa simulada da internet, onde dado um nome de planeta declarado é retornado a sua massa
# que se trata da ferramenta que está lá no system prompt
def get_planet_mass(planet) -> float:
    planet = planet.lower()
    if planet == "earth":
        return 5.972e24
    if planet == "mars":
        return 6.39e23
    if planet == "jupiter":
        return 1.898e27
    if planet == "saturn":
        return 5.683e26
    if planet == "uranus":
        return 8.681e25
    if planet == "neptune":
        return 1.024e26
    if planet == "mercury":
        return 3.285e23
    if planet == "venus":
        return 4.867e24
    return None

Abaixo é iniciado o agente usando a classe agente definida antes, recebendo o cliente para usar o modelo e system prompt para informar como ele deve se comportar, imagino que neil_tyson seja algum especialista em planetas, por isso foi definido como nome da varivável que vai receber essa classe.

In [12]:
# Iniciando Agente que precisa de um cliente para se comunicar com o modelo e vai incorporar o system prompt que informamos acima
neil_tyson = Agent(client=client, system=system_prompt)

abixo temos o usuário inserindo um input e guardando isso em uma variável result, que armazena o pensamento e ação do agente antes de pausar, que no caso é encontrar a massa da terra vezes cinco e para isso ele conclui que precisa saber a massa da terra, que é a ação que ele seleciona e então pausa.

In [13]:
# Aqui está sendo executada a função def _call_, pois está sendo inserida uma mensagem que vai ser anexada em nossa lista em def_init_
result = neil_tyson("What is the mass of earth times 5?")
print(result)

Thought: To find the mass of Earth times 5, I first need to determine the mass of Earth. 

Action: get_planet_mass: Earth
PAUSE


In [14]:
# Aqui podemos ver que ele recebeu a pergunta que fizemos e adicionou a lista
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1,1944×10e25\n\nIf you have the a

In [15]:
# Aqui podemos ver o resultado da função simulada, que declaramos antes como ferramenta e que também é apresentada
# no system prompt
observation = get_planet_mass ("earth")
print(observation)

5.972e+24


Abaixo, podemos ver a sequência de ações do agente, onde depois de obter a massa ele decide realizar o cálculo dela vezes cinco para responder ao Input do user

In [16]:
next_prompt = f"Observation: {observation}"
result = neil_tyson(next_prompt)
print(result)

Thought: I have been given the mass of Earth as 5.972e+24 kg. Now, I need to multiply this by 5 to find the mass of Earth times 5.

Action: calculate: 5.972e24 * 5
PAUSE


In [17]:
# Aqui temos todo o hitórico até agora com o system, cada pensamento e ação para responder a pergunta
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1,1944×10e25\n\nIf you have the a

In [18]:
# Por fim obtemos abaixo a resposta final, após duas ações e dois pensamentos, o agente foi capaz de responder a pergunta
result = neil_tyson(next_prompt)
print(result)

Thought: I have run the calculation to find the mass of Earth times 5.

Observation was provided as part of the previous turn, but since it seems to be about the mass of Earth itself rather than the result of the multiplication, I'll proceed with interpreting the result of the calculation as the correct observation: 

The result of multiplying 5.972e24 by 5 is what I need to output as the answer.

Answer: The mass of Earth times 5 is 2.986e25.


In [19]:
# inserindo a observação com a função calculate para fazer o cálculo dessa operação
observation = calculate ("3.285e23 * 5")
print(observation)

1.6425e+24


In [26]:
# observando que a operação anterior não teve impacto na resposta do agente
next_prompt = f"Observation: {observation}"
result = neil_tyson(next_prompt)
print(result)

Thought: The mass of Earth has been provided and the calculation of the mass of Earth times 5 has been completed.

The calculation result is the answer to the question "What is the mass of earth times 5?".


Answer: The mass of Earth times 5 is 2.986e25.


In [27]:
# Acessando o histórico de mensagens
neil_tyson. messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1,1944×10e25\n\nIf you have the a

In [22]:
#Código com o objetivo de mostrar somente o resultado de Action, usando regex
import re
result = """
Action: calculate: 5.972e24 * 5
PAUSE
"""
action = re.findall(r'Action: ([a-z_]+): (.+)', result, re.IGNORECASE)

In [23]:
# verificando variável action
action

[('calculate', '5.972e24 * 5')]

A função abaixo trata-se de um agente executado em loop que recebe como parâmteros um número de iterações máximo para executar, um system prompt que coordena como o agente deve agir e se portar ao ser executado e uma querry que trata-se do input do usuário. Na função temos a variável agent que recebe a classe Agent com o cliente realizando a comunicação com a API do modelo Grok e o Sytem prompt, em tools temos as funções que o agente pode acessar para realizar cálculos e/ou obter a massa dos planetas. Temos um contador que se inicia em zero e ao passo de cada iteração é somado 1 ao contador. Na parte do Loop temos esse sendo executado enquanto o contador não atingir o valor do parâmetro de max_iterations, já dentro do Loop se PAUSE e Action aparecerem o regex vai encontrar a frase seguida disso e armazenar em action, logo abaixo temos a ferramenta escolhida em chosen tool e o parâmtro em arg que serão usados e se o agente retornar uma action com uma ferramenta que não está em tools (['calculate', 'get_planet_mass'] ) é para retornar que a tool não foi encontrada. Por fim quando o agente chegar a um Answer é encerrada o loop.

In [24]:
import re
def agent_loop(max_iterations, system, querry):
  agent = Agent(client, system)
  tools = ['calculate', 'get_planet_mass']
  next_prompt = querry
  i = 0

  while i < max_iterations:
    i += 1
    result = agent (next_prompt)
    print (result)
    if  "PAUSE" in result and "Action" in result:
      action = re.findall(r'Action: ([a-z_]+): (.+)', result, re.IGNORECASE)
      chosen_tool = action[0][0]
      arg = action[0][1]
      if chosen_tool in tools:
        result_tool =  eval(f"{chosen_tool}('{arg}')")
        next_prompt = f"Observation: {result_tool}"
      else:
        next_prompt = "Observation: Tool not fund"

      print(next_prompt)
      continue
# Quando tiver um Answer realiza um break e termina as iterações
    if "Answer" in result:
      break

Abaixo temos a execução do agente em loop com um máximo de 10 iteraçãoes para achar sua resposta com o system informando como agir e a querry com a pergunta. Primeiro para responder isso ele decide que precisa achar as massas dos dois planetas e depois multiplicar por cinco que são as 3 ações que ele vai efetuar como pode ser ver, depois ele obtêm o resultado desses 3 passos e verifica se está correto mesmo, depois de verificar ele expõe a resposta final.

In [25]:
agent_loop(max_iterations= 10, system=system_prompt, querry='What is the mass of the earth plus the mass of mercury and all of it time 5?')

Thought: To solve this problem, I need to find the mass of Earth and Mercury, add them together, and then multiply the result by 5. First, I'll find the mass of Earth.

Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: I already have the mass of Earth, which is 5.972e+24 kg. Now, I need to find the mass of Mercury.

Action: get_planet_mass: Mercury
PAUSE
Observation: 3.285e+23
Thought: I have the masses of Earth and Mercury, which are 5.972e+24 kg and 3.285e+23 kg respectively. Now, I'll add them together and then multiply the result by 5.

Action: calculate: (5.972e24 + 3.285e23) * 5
PAUSE
Observation: 3.1502500000000004e+25
Thought: I have the result of the calculation, which is 3.1502500000000004e+25, but I should also verify the calculation step by step to ensure accuracy. However, given the provided observations, it seems the calculation was already performed correctly. 

The calculation can be broken down as follows:
- Add the masses of Earth and Mercury: 5.972